# Aluno: Giseldo da Silva Neo

## Questão 1
Utilize o k-means para procurar grupos de músicas, identificando o melhor número de grupos através das técnicas explicadas nas aulas. Em seguida, descreva os grupos encontrados, listando as palavras e os artistas mais frequentes em cada grupo. Tente nomear os grupos e explique em detalhes o racional para sua nomeação para cada grupo. Repita este processo utilizando como entrada para o k-means:
   O título da música;
   A letra da música;
   A concatenação do título e letra;

## Respostas 1

Para a Letra da música foram 9 grupos.

P: Utilize o k-means para procurar grupos de músicas, identificando o melhor número de grupos através das técnicas explicadas nas aulas. 

R:  A melhor quantidade de grupos foi 9 com erro (SSD)  de 7,48

P: Em seguida, descreva os grupos encontrados, listando as palavras e os artistas mais frequentes em cada grupo. Tente nomear os grupos e explique em detalhes o racional para sua nomeação para cada grupo. 

R: O grupo 0 é chamado de **Grandioso**, por ter palavras como tudo, todo etc

R: Palavras do Grupo 0: todo mundo, mundo, todo, baby, pra, ninguém, tudo, dia, tchu, bem, quer, fácil, sabe, vida

R: Artistas do grupo 0: Lulu Santos, Engenheiros do Hawaii, Jota Quest, Blitz, Barão Vermelho, Velhas Virgens, Lobão, Rita Lee, Charlie Brown Jr, Erasmo Carlos,

R: O grupo 1 é chamado de **Andarilho**, porque tem palavras como vai, gente, vem etc

R: Palavras do Grupo 1: vai, gente, pra, tudo, bem, vamos, vida, dia, faz, quer, ninguém, sempre, vem, pode

R: Artistas do Grupo 1: Lulu Santos, Engenheiros do Hawaii, Raimundos, Skank, Charlie Brown Jr, Jota Quest, Fresno, Velhas Virgens, Rita Lee, Pato Fu

R: O grupo 2 é chamado de **Romantico**, porque tem palavras de amor, coração etc

R: Palavras do Grupo 2: amor, pra, mim, tudo, coração, quero, vida, vou, sol, dor, vem, bem, vai, assim

R: Artistas do Grupo 2: Jota Quest, Lulu Santos, Erasmo Carlos, Skank, Rita Lee, Los Hermanos, Cássia Eller, Barão Vermelho, Cazuza, Os Paralamas do Sucesso

R: O grupo 3 é chamado de **Rotação**, porque fala sobre sol, dia, niote etc

R: Palavras do Grupo 3: nada, pra, dia, faz, vida, quer, tanto, aqui, tudo, sei, deus, noite, quero, luz

R: Artistas do Grupo 3: Engenheiros do Hawaii, Lulu Santos, Rita Lee, Titãs, Cássia Eller, Raul Seixas, Skank, Erasmo Carlos, Capital Inicial, Os Paralamas do Sucesso    

R: O grupo 4 é chamado de **Grudento**, porque tem palavas como querto, tudo etc

R: Palavras do Grupo 4: quero, bem, mim, sei, pra, tão, sempre, tudo, vem, ver, assim, vezes, sinto, pode

R: Artistas do Grupo 4: Lulu Santos, Engenheiros do Hawaii, Skank, Erasmo Carlos, Cássia Eller, Charlie Brown Jr, Ira!, Rita Lee, Biquini Cavadão, Capital Inicial,

R: O grupo 5 é chamado de **Pé no chão**, porque tem palavras como hoje, agora etc

R: Palavras do Grupo 5: pra, tempo, tudo, vida, onde, nunca, mim, nada, ter, dia, viver, sei, agora, hoje

R: Artistas do Grupo 5: Engenheiros do Hawaii, Fresno, CPM 22, Nx Zero, Skank, Jota Quest, Lulu Santos, Rosa de Saron, Dead Fish, Capital Inicial,

R: O grupo 6 é chamado de **Americano**, porque tem palavras em ingles etc

R: Palavras do Grupo 6: you, the, and, your, love, that, all, know, don, get, now, from, waiting, waiting you

R: Artistas do Grupo 6: Jota Quest, Os Paralamas do Sucesso, Rita Lee, Nx Zero, Blitz, Mutantes, Charlie Brown Jr, Dead Fish, Skank, Rosa de Saron

R: O grupo 7 é chamado de **Sabe-Tudo**, porque tem palavras de sei e vou etc

R: Palavras do Grupo 7: vou, pra, quero, vai, mim, sei, onde, tudo, aqui, deus, hoje, mundo, ficar, vou deixar

R: Artistas do Grupo 7: Jota Quest, Skank, Lulu Santos, Titãs,  Cássia Eller, Charlie Brown Jr, CPM 22, Strike, Fresno, Detonautas

R: O grupo 8 é chamado de **Felicidade**, porque tem palavras tipo feliz, bela etc

R: Palavras do Grupo 8:  feliz, dia, feliz feliz, vida, feliz aniversário, tudo, primavera, faz, aniversário, diz, fiz, bela, pra, hoje

R: Artistas do Grupo 8: Cássia Eller, Aliados, Jota Quest,  Charlie Brown Jr, Rita Lee, Biquini Cavadão, Los Hermanos, Nx Zero, Barão Vermelho, Leoni

R: Para o título da música foram 5 Grupos

R: O grupo 0 é chamado de **Tempo**

R: 0: pra, vida, mundo, part, tempo, dia, mim, nada, sol, homem, vai, noite, coração, vou

R: 0: Jota Quest, Engenheiros do Hawaii, Lulu Santos, Skank, Charlie Brown Jr, Detonautas, Erasmo Carlos, Rita Lee, Barão Vermelho, Titãs

R: O grupo 1 é chamado de **Romantico**

R: 1: amor, apenas, apenas amor, maior, grande amor, grande, vítima, carta, balada, canção, verdadeiro, quente, morrer amor, melô

R: 1: Lulu Santos, Blitz, Leoni, Jota Quest, Rita Lee, PG, Ira!, Barão Vermelho, Engenheiros do Hawaii, Skank,

R: O grupo 2 é chamado de **Inferno**

R: 2: país, diabo, voltar, nando, levar, poeira, esquecer, cruel, planeta, vão, canções, gosta, semana, ouro

R: 2: Lulu Santos,  Engenheiros do Hawaii,  Rita Lee, Erasmo Carlos,  Skank, Titãs,  Cássia Eller, Capital Inicial,  Raul Seixas, Os Paralamas do Sucesso,

R: O grupo 3 é chamado de **Cosmico**

R: 3: dois, feat, senhor, céu, longe, cruz, don, mil, boca, universo, maria, baby, rei, guerra

R: 3: Engenheiros do Hawaii, Lulu Santos, Skank, Cássia Eller,  Jota Quest, Rita Lee,  Blitz, Erasmo Carlos, Charlie Brown Jr, Raul Seixas,

R: O grupo 4 é chamado de **Mudança**

R: 4: tudo, tudo bem, tudo vai, azul, bem, vai, lembrar, faz, parado, fim, igual, feito, mudar, passa

R: 4: Lulu Santos, Jota Quest, Frejat, Mutantes, Charlie Brown Jr, Malta, Lobão, Capital Inicial, Nx Zero, Matanza

R: Para o título e letra juntos foram 3 Grupos

R: Grupo 0:

R: O grupo 4 é chamado de **Romantico**

R: 0: amor, quero, pra, bem, mim, coração, sei, tudo, vem, vida, sempre, tão, assim, ver

R: 0: Lulu Santos, Erasmo Carlos, Jota Quest, Rita Lee, Cássia Eller, Skank, Blitz, Barão Vermelho, Isabella Taviani, Raul Seixas

R: Grupo 1:

R: O grupo 4 é chamado de **Mudança**

R: 1: pra, vai, tudo, vida, dia, gente, mundo, nada, tempo, sei, mim, sempre, aqui, bem

R: 1: Engenheiros do Hawaii, Lulu Santos, Skank, Rita Lee, Capital Inicial, Jota Quest, Titãs, Charlie Brown Jr, Raul Seixas, Cássia Eller         

R: Grupo 2:

R: O grupo 4 é chamado de **Sempre em Frente**

R: 2: vou, pra, onde, vai, sei, mim, tudo, quero, aqui, amor, ficar, sempre, vem, vida

R: 2: Jota Quest, Lulu Santos, Strike, Skank, Charlie Brown Jr, Fresno, CPM 22, Cássia Eller, Titãs, Malta

## Questão 2
P: Mesmo que 1 acima, mas aplique uma redução de dimensionalidade utilizando o PCA, e utilize os dados transformados como entrada para o k-means. 

R: Não foi possível utilizar o PCA (TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.) portanto foi utilizado o TruncateSVD para essa redução.

## questão 3
P: Compare a qualidade dos grupos com e sem PCA usando o coeficiente de silhueta.

R: Comparando a qualidade dos grupos percebe-se que o coeficiente de silhueta é melhor (0.0012 contra 0.189) quando aplicado a redução de dimensionalidade.

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import kmeans_plusplus
from nltk.corpus import stopwords
import nltk
import re
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import TSNE

In [3]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gisel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Foi escolhido as musicas de rock

In [6]:
letras = pd.read_csv("letras-ptbr-rock-grande.csv")
letras.head()

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
0,Esperança,Enquanto ela estiver aqui. Ainda haverá o amor...,Aliados,75,0.8,Rock,Rock; Pop/Rock; Rock Alternativo; Surf Music; ...
1,Te Encontro Por Aí,Me deu a luz. Me fez sonhar. Chorou por mim. M...,Aliados,75,0.8,Rock,Rock; Pop/Rock; Rock Alternativo; Surf Music; ...
2,Pra Ser Feliz,"Nasce uma criança, tem que acreditar. Que a fo...",Aliados,75,0.8,Rock,Rock; Pop/Rock; Rock Alternativo; Surf Music; ...
3,Águas Passadas,Deixa rolar. Deixa fluir. Só não deixe de apre...,Aliados,75,0.8,Rock,Rock; Pop/Rock; Rock Alternativo; Surf Music; ...
4,No Seu Coração,"Para, pensa. Onde está seu verdadeiro amor. E ...",Aliados,75,0.8,Rock,Rock; Pop/Rock; Rock Alternativo; Surf Music; ...


In [7]:
letras.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
3046,"Robô, Robôa","De repente, você reapareceu. Está misto dilere...",Lobão,112,1.3,Rock,Rock; MPB; Rock Alternativo; Punk Rock; Psicod...
3254,Desculpe o Auê,Desculpe o auê. Eu não queria magoar você. Foi...,Lulu Santos,271,10.0,Rock,Pop/Rock; Rock; Pop; MPB; Dance; Electronica; ...
728,All Star,Estranho seria se eu não me apaixonasse por vo...,Cássia Eller,141,7.5,Rock,Rock; MPB; Pop/Rock; Pop; Romântico; Samba; Co...
3271,Esse Brilho Em Teu Olhar,Eu te procuro quando me sinto em perigo. Quand...,Lulu Santos,271,10.0,Rock,Pop/Rock; Rock; Pop; MPB; Dance; Electronica; ...
3229,Cara Legal,E nesse filme eu quero ser mocinho. E conquist...,Lulu Santos,271,10.0,Rock,Pop/Rock; Rock; Pop; MPB; Dance; Electronica; ...
54,Fala Pra Ela,Lá vem o sol depois de tudo que eu falei. se a...,Aliados,75,0.8,Rock,Rock; Pop/Rock; Rock Alternativo; Surf Music; ...
5203,Perigosa,Sei que eu sou bonita e gostosa. E sei que voc...,Rita Lee,297,7.6,Rock,Rock; Pop/Rock; MPB; Rock Alternativo; Jovem G...
1705,Olhos iguais aos seus,Uma nuvem cobre o céu. Uma sombra envolve o se...,Engenheiros do Hawaii,193,11.9,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Se...
979,Contrastes da Vida,A ação do tempo nas nossas vidas. Deixou a gen...,Charlie Brown Jr,208,25.7,Rock,Pop/Rock; Rap; Rock; Reggae; Rock Alternativo;...
558,A Saga do Homo Babaca,Quando um homem. Caminha pelo mundo. Em busca ...,Capital Inicial,196,8.1,Rock,Pop/Rock; Rock; Rock Alternativo; Punk Rock; M...


In [8]:
letras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7680 entries, 0 to 7679
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SName       7680 non-null   object 
 1   Lyric       7680 non-null   object 
 2   Artist      7680 non-null   object 
 3   Songs       7680 non-null   int64  
 4   Popularity  7680 non-null   float64
 5   Genre       7680 non-null   object 
 6   Genres      7680 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 420.1+ KB


In [9]:
letras["Genre"].value_counts()

Rock    7680
Name: Genre, dtype: int64

São 7680 Musicas de rock nesse CSV

In [10]:
letras["Artist"].value_counts()[0:20]

Lulu Santos                462
Engenheiros do Hawaii      402
Jota Quest                 296
Skank                      290
Rita Lee                   282
Erasmo Carlos              236
Cássia Eller               226
Titãs                      209
Charlie Brown Jr           198
Raul Seixas                196
Capital Inicial            195
Biquini Cavadão            184
Os Paralamas do Sucesso    181
Barão Vermelho             164
Velhas Virgens             156
Fresno                     153
Blitz                      150
Pato Fu                    146
Ira!                       142
Rosa de Saron              137
Name: Artist, dtype: int64

Quem tem mais música é o Lulu Santos, Rita Lee, Erasmo Carlos e outros.

Fazer a partir do títulos

## Pré-processamento

In [11]:
stop_words = set(stopwords.words("portuguese"))
print(len(stop_words))
stop_words

207


{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estar',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houverí

As stopwords são palavras comuns que não ajudam aos algoritmos. 

Serão removidos asstops words do título, da letra e do título mais a letra das músicas.

Remover Stopwords do Título

In [12]:
# remover do titulo
clean_SName = []
for w in range(len(letras.Lyric)):
    SName = letras["SName"][w]

    # remove special characteres and digits
    SName = re.sub("(\\d|\\W)+|\w*\d\w*", " ", SName)
    SName = ' '.join(s for s in SName.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_SName.append(SName)

clean_SName[0:5]

['Esperança', 'Encontro Por', 'Pra Ser Feliz', 'Águas Passadas', 'Seu Coração']

Remover StopWords do Título e do Lyric Juntos

In [13]:
letras["SNameLyric"] = letras['SName'] + " " + letras['Lyric']
clean_SName_Lyrics = []
for w in range(len(letras.SNameLyric)):
    SNameLyric = letras["SNameLyric"][w]

    # remove special characteres and digits
    SNameLyric = re.sub("(\\d|\\W)+|\w*\d\w*", " ", SNameLyric)
    SNameLyric = ' '.join(s for s in SNameLyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_SName_Lyrics.append(SNameLyric)

clean_SName_Lyrics[0:2]

['Esperança Enquanto ela estiver aqui Ainda haverá amor Com ela estou feliz com ela enfrento dor Não adianta fugir não adianta chorar dia ela sumir nada mais irá sobrar Sonhar viver todo dia agradecer rezar pra você ser última morrer Podem tentar atingir Podem mandar pra onde for Enquanto ela estiver aqui vida ainda tem valor Sonhar viver todo dia agradecer rezar pra você ser última morrer Uma gota você vale mais que tudo Quando solução não pode resolver fico com você esperança fico com você Sonhar viver todo dia agradecer rezar pra você ser última morrer Sonhar viver todo dia agradecer rezar pra você ser última morrer esperança Esperança',
 'Encontro Por deu luz fez sonhar Chorou por mim fez amar Foi pra mim Foi tudo mais pouco enfim Canto quando sinto sua falta Rezo pelo bem sua alma queria que você vivesse mais vida levou deixou aqui que você ensinou que faz feliz Que Deus bom lugar guarde pra mim quando for minha vez encontro por Fale mais Quero ouvir Onde está Não vejo mas pra sen

Remover stopwords da letra

In [14]:
clean_lyrics = []
for w in range(len(letras.Lyric)):
    lyric = letras["Lyric"][w]

    # remove special characteres and digits
    lyric = re.sub("(\\d|\\W)+|\w*\d\w*", " ", lyric)
    lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_lyrics.append(lyric)

clean_lyrics[0:3]

['Enquanto ela estiver aqui Ainda haverá amor Com ela estou feliz com ela enfrento dor Não adianta fugir não adianta chorar dia ela sumir nada mais irá sobrar Sonhar viver todo dia agradecer rezar pra você ser última morrer Podem tentar atingir Podem mandar pra onde for Enquanto ela estiver aqui vida ainda tem valor Sonhar viver todo dia agradecer rezar pra você ser última morrer Uma gota você vale mais que tudo Quando solução não pode resolver fico com você esperança fico com você Sonhar viver todo dia agradecer rezar pra você ser última morrer Sonhar viver todo dia agradecer rezar pra você ser última morrer esperança Esperança',
 'deu luz fez sonhar Chorou por mim fez amar Foi pra mim Foi tudo mais pouco enfim Canto quando sinto sua falta Rezo pelo bem sua alma queria que você vivesse mais vida levou deixou aqui que você ensinou que faz feliz Que Deus bom lugar guarde pra mim quando for minha vez encontro por Fale mais Quero ouvir Onde está Não vejo mas pra sentir Foi pra mim Foi tud

# Vetores TF-IDF do título

In [15]:
# TF - IDF vectorizer
tfv = TfidfVectorizer(
    min_df = 5,
    max_df = 0.9,
    max_features = None,
    stop_words = stop_words,
    ngram_range = (1, 3)
)

# transform
vec_text = tfv.fit_transform(clean_SName)

words = tfv.get_feature_names()

len(words)

677

In [16]:
words[1:10]

['acabar',
 'acaso',
 'acima',
 'acústico',
 'acústico mtv',
 'adeus',
 'admirável',
 'adão',
 'adão eva']

Decidindo K

In [17]:
qualidade = pd.DataFrame(columns = ['k', 'ssd'])
for k in range(1, 10, 1):
    # kemans = KMeans(n_clusters=k, rantom_state=0)
    kmeans = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)
    kmeans.fit(vec_text)
    qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)

alt.Chart(qualidade).mark_line(
    point=True
).encode(
    x = 'k',
    y = alt.Y('ssd', scale = alt.Scale(zero=False))
)

alt.Chart(...)

Foi escolhido 5 grupos a partir dos títulos

In [18]:
# setup kmeans clustering
kmeans = MiniBatchKMeans(n_clusters=5, init_size=1024, batch_size=2048, random_state=20)

# fit the data
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

letras_ag = letras.assign(grupo = labels)

In [26]:
print("Coeficiente de Silhueta K-Means: %0.3f" % metrics.silhouette_score(vec_text, labels,metric='euclidean'))

Coeficiente de Silhueta K-Means: 0.229


In [27]:
import numpy as np
import pandas as pd 
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist
from scipy import linalg
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn import metrics
import seaborn as sns
from sklearn.preprocessing import StandardScaler
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 10)

In [29]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np

In [19]:
letras_ag.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres,SNameLyric,grupo
2237,Logo Você,A cada esquina teimo ouvir. sua voz a me lembr...,Fresno,142,2.2,Rock,Pop/Rock; Emocore; Rock; Pop/Punk; Hardcore; P...,Logo Você A cada esquina teimo ouvir. sua voz ...,0
5915,Garota Nacional,Aqui nesse mundinho fechado ela é incrível. Co...,Skank,139,12.3,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Ro...,Garota Nacional Aqui nesse mundinho fechado el...,0
4389,Sorte e Azar,Tudo está. Fora de seu lugar. Já notei. O mund...,Pato Fu,146,1.4,Rock,Rock; Pop/Rock; Pop; MPB; Indie; Rock Alternat...,Sorte e Azar Tudo está. Fora de seu lugar. Já ...,0
247,Sombras No Escuro,Escuro.... Medo de abrir os olhos. Não ver que...,Barão Vermelho,160,3.8,Rock,Rock; Pop/Rock; MPB; Romântico; Rock Alternati...,Sombras No Escuro Escuro.... Medo de abrir os ...,0
7580,Beleza Pura,Não me amarra dinheiro não. Mas formosura. Din...,Skank,139,12.3,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Ro...,Beleza Pura Não me amarra dinheiro não. Mas fo...,0
3850,O Que Sobrou do Céu,Faltou luz mas era dia. O sol invadiu a sala. ...,O Rappa,97,10.2,Rock,Reggae; Rap; Rock; Pop/Rock; Black Music; Funk...,O Que Sobrou do Céu Faltou luz mas era dia. O ...,0
1566,Um Girassol da Cor de Seu Cabelo,Vento solar e estrelas do mar. A terra azul da...,Emmerson Nogueira,96,0.9,Rock,Rock; Pop/Rock; Pop; Romântico; MPB; World Mus...,Um Girassol da Cor de Seu Cabelo Vento solar e...,0
940,Um Dia a Gente Se Encontra,Um dia na praia a gente jurou. Ficar junto pra...,Charlie Brown Jr,208,25.7,Rock,Pop/Rock; Rap; Rock; Reggae; Rock Alternativo;...,Um Dia a Gente Se Encontra Um dia na praia a g...,0
4500,Ah! Ah! Ah!,"Eu gosto é de rir, gargalhar me faz tão bem.. ...",PG,111,1.1,Rock,Rock; Gospel/Religioso; Pop/Rock; Funk; Clássi...,"Ah! Ah! Ah! Eu gosto é de rir, gargalhar me fa...",0
7621,Minas Com Bahia,Sacudir estrelas. Despertar desejos. Numa noit...,Skank,139,12.3,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Ro...,Minas Com Bahia Sacudir estrelas. Despertar de...,0


In [20]:
letras_ag['grupo'].value_counts()

0    7350
1     176
4     106
3      40
2       8
Name: grupo, dtype: int64

In [21]:
# this loop transform the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-15:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : pra, vida, mundo, part, tempo, dia, mim, homem, nada, sol, vai, coração, noite, vou
1 : amor, apenas, apenas amor, maior, vítima, grande amor, grande, balada, morrer amor, carta, morrer, canção, quente, leva
2 : ilha, todo mundo, todo, mundo, único, feat, festa, feliz, felicidade, feito, feira, fazer, esquecer sunrise, faz parte
3 : dois, paraíso, passos, olhos, bom, mil, amores, selva, rua, samba, pra, resto, sobre, part
4 : tudo, tudo bem, azul, bem, parado, tudo vai, lembrar, faz, igual, vai, fim, feito, meio, longe


In [22]:
pd.options.display.max_colwidth = 100
letras_ag.query('grupo == 0')[["SName", "Lyric", "Artist"]].sample(10)

,SName,Lyric,Artist
1615,Anjo da Guarda,Anjo Da Guarda. Engenheiros do Hawaii. Composição: Humberto Gessinger. Sem que você saiba. Há um...,Engenheiros do Hawaii
5744,Eterno Para Você,Sem você eu cheguei a pensar. Em não mais viver. Vou sangrar até morrer. Se você não ficar. Eu s...,Saga Crepúsculo
196,Não Quero Seu Perdão,A gente nasce. A gente cresce. E vive. O tempo todo perto dela. E é tão bonita. Essa razão. Tão ...,Barão Vermelho
6085,Amianto,"Moça, sai da sacada. Você é muito nova pra brincar de morrer. Me diz o que há,. O quê que a vida...",Supercombo
2078,A Felicidade Bate À Sua Porta,"O trem da alegria saiu agora. Partiu nesse instante. Da rádio nacional,. A gare principal da cen...",Frejat
235,Rock do Cachorro Morto,"Lembro-me que certo dia. Na rua, ao sol de um verão. Envenenado morria, um pobre cão. Arfava, es...",Barão Vermelho
597,Despertar Dos Mortos,Roubaram meu ouro. Roubaram meu sangue. Roubaram meus filhos. E querem mais do. Verde e Amarelo....,Capital Inicial
7216,Grito,"E AGORA, FAZER O QUÊ COM A LIBERDADE?. DESORGANIZAR O QUÊ. NO MEIO DESTA CONFUSÃO?. AS LUZES QUE...",Jota Quest
7271,Sexo e Paixão (Feat. Mista Raja),"Ela faz com amor nega love, beijando a boca. Ela faz com paixão nega hot, tirando a roupa. Ela f...",Jota Quest
4767,Procedência C.d,Venha ver um novo parque de diversão. Nele há brinquedos q aumentam a percepção. Visão privilegi...,Planet Hemp


In [24]:
for g in range(0, 5):
    print("\n-----\nGrupo {}:".format(g))
    print(letras_ag.query('grupo == {}'.format(g))["Artist"].value_counts()[0:10])
    print("-----")


-----
Grupo 0:
Lulu Santos              432
Engenheiros do Hawaii    390
Skank                    278
Jota Quest               272
Rita Lee                 271
Erasmo Carlos            228
Cássia Eller             220
Titãs                    201
Raul Seixas              194
Charlie Brown Jr         193
Name: Artist, dtype: int64
-----

-----
Grupo 1:
Lulu Santos              14
Blitz                    12
Leoni                    12
Jota Quest               12
Rita Lee                  8
PG                        8
Ira!                      7
Barão Vermelho            7
Engenheiros do Hawaii     6
Skank                     6
Name: Artist, dtype: int64
-----

-----
Grupo 2:
Engenheiros do Hawaii    4
Pato Fu                  2
Dead Fish                1
Raul Seixas              1
Name: Artist, dtype: int64
-----

-----
Grupo 3:
Frejat           4
Blitz            4
Skank            4
Los Hermanos     3
Aliados          2
RPM              2
Jota Quest       2
Malta            2
Erasmo 

In [25]:
letras_embedded = TSNE(n_components=2, verbose=1, perplexity=45).fit_transform(vec_text)

letras_ag = letras_ag.assign(tsnel = letras_embedded[:,0], tsne2 = letras_embedded[:,1])

alt.Chart(letras_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = "tsnel",
    y = "tsne2",
    color = "grupo:N",
    tooltip = ["Artist", "SName"]
).interactive()

[t-SNE] Computing 136 nearest neighbors...
[t-SNE] Indexed 7680 samples in 0.001s...
[t-SNE] Computed neighbors for 7680 samples in 2.478s...
[t-SNE] Computed conditional probabilities for sample 1000 / 7680
[t-SNE] Computed conditional probabilities for sample 2000 / 7680
[t-SNE] Computed conditional probabilities for sample 3000 / 7680
[t-SNE] Computed conditional probabilities for sample 4000 / 7680
[t-SNE] Computed conditional probabilities for sample 5000 / 7680
[t-SNE] Computed conditional probabilities for sample 6000 / 7680
[t-SNE] Computed conditional probabilities for sample 7000 / 7680
[t-SNE] Computed conditional probabilities for sample 7680 / 7680
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 97.147560
[t-SNE] KL divergence after 1000 iterations: 1.479277


alt.Chart(...)

# Vetores TF-IDF da Letra

In [24]:
# TF - IDF vectorizer
tfv = TfidfVectorizer(
    min_df = 5,
    max_df = 0.9,
    max_features = None,
    stop_words = stop_words,
    ngram_range = (1, 3)
)

# transform
vec_text = tfv.fit_transform(clean_lyrics)

words = tfv.get_feature_names()

len(words)

c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


22581

In [25]:
words[1:10]

['aah',
 'aba',
 'abafada',
 'abaixa',
 'abaixe',
 'abaixo',
 'abajur',
 'abala',
 'abalar']

In [26]:
qualidade = pd.DataFrame(columns = ['k', 'ssd'])
for k in range(1, 5, 1):
    # kemans = KMeans(n_clusters=k, rantom_state=0)
    kmeans = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)
    kmeans.fit(vec_text)
    qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)

alt.Chart(qualidade).mark_line(
    point=True
).encode(
    x = 'k',
    y = alt.Y('ssd', scale = alt.Scale(zero=False))
)

C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\3098802002.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\3098802002.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\3098802002.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\3098802002.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

alt.Chart(...)

In [27]:
# setup kmeans clustering
kmeans = MiniBatchKMeans(n_clusters=3, init_size=1024, batch_size=2048, random_state=20)

# fit the data
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

letras_ag = letras.assign(grupo = labels)

In [28]:
# this loop transform the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-15:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : quero, amor, pra, bem, mim, coração, sei, tudo, vida, tão, vem, faz, ver, deus
1 : pra, vai, tudo, vida, dia, gente, tempo, nada, mundo, sei, sempre, mim, tão, aqui
2 : vou, pra, onde, vai, sei, mim, tudo, quero, amor, aqui, vem, ficar, agora, sempre


In [29]:
for g in range(0, 3):
    print("\n-----\nGrupo {}:".format(g))
    print(letras_ag.query('grupo == {}'.format(g))["Artist"].value_counts()[0:10])
    print("-----")


-----
Grupo 0:
Rita Lee          130
Lulu Santos       118
Erasmo Carlos     101
Cássia Eller       80
Jota Quest         76
Skank              72
Blitz              64
Raul Seixas        60
Velhas Virgens     51
Titãs              51
Name: Artist, dtype: int64
-----

-----
Grupo 1:
Engenheiros do Hawaii      350
Lulu Santos                290
Skank                      174
Jota Quest                 162
Capital Inicial            151
Charlie Brown Jr           138
Biquini Cavadão            136
Os Paralamas do Sucesso    130
Titãs                      128
Rita Lee                   128
Name: Artist, dtype: int64
-----

-----
Grupo 2:
Jota Quest          58
Lulu Santos         54
Strike              44
Skank               44
Charlie Brown Jr    40
Cássia Eller        36
Fresno              34
Detonautas          31
Titãs               30
CPM 22              29
Name: Artist, dtype: int64
-----


In [30]:
letras_embedded = TSNE(n_components=2, verbose=1, perplexity=45).fit_transform(vec_text)

letras_ag = letras_ag.assign(tsnel = letras_embedded[:,0], tsne2 = letras_embedded[:,1])

alt.Chart(letras_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = "tsnel",
    y = "tsne2",
    color = "grupo:N",
    tooltip = ["Artist", "SName"]
).interactive()

[t-SNE] Computing 136 nearest neighbors...
[t-SNE] Indexed 7680 samples in 0.003s...


c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computed neighbors for 7680 samples in 2.980s...
[t-SNE] Computed conditional probabilities for sample 1000 / 7680
[t-SNE] Computed conditional probabilities for sample 2000 / 7680
[t-SNE] Computed conditional probabilities for sample 3000 / 7680
[t-SNE] Computed conditional probabilities for sample 4000 / 7680
[t-SNE] Computed conditional probabilities for sample 5000 / 7680
[t-SNE] Computed conditional probabilities for sample 6000 / 7680
[t-SNE] Computed conditional probabilities for sample 7000 / 7680
[t-SNE] Computed conditional probabilities for sample 7680 / 7680
[t-SNE] Mean sigma: 0.546143
[t-SNE] KL divergence after 250 iterations with early exaggeration: 121.273170
[t-SNE] KL divergence after 1000 iterations: 3.780425


c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

# Vetores TF-IDF do título e da letra

In [31]:
tfv = TfidfVectorizer(
    min_df = 5,
    max_df = 0.9,
    max_features = None,
    stop_words = stop_words,
    ngram_range = (1, 3)
)

# transform
vec_text = tfv.fit_transform(clean_SName_Lyrics)

words = tfv.get_feature_names()

len(words)

c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


22864

In [32]:
from sklearn.cluster import MiniBatchKMeans


qualidade = pd.DataFrame(columns = ['k', 'ssd'])
for k in range(1, 10, 1):
    # kemans = KMeans(n_clusters=k, rantom_state=0)
    kmeans = MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20)
    kmeans.fit(vec_text)
    qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)

alt.Chart(qualidade).mark_line(
    point=True
).encode(
    x = 'k',
    y = alt.Y('ssd', scale = alt.Scale(zero=False))
)

C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\1068257517.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\1068257517.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\1068257517.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qualidade = qualidade.append({'k': k, 'ssd': kmeans.inertia_}, ignore_index=True)
C:\Users\gigi\AppData\Local\Temp\ipykernel_11992\1068257517.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

alt.Chart(...)

In [33]:
# setup kmeans clustering
kmeans = MiniBatchKMeans(n_clusters=9, init_size=1024, batch_size=2048, random_state=20)

# fit the data
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

letras_ag = letras.assign(grupo = labels)

In [34]:
letras_ag.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres,SNameLyric,grupo
1301,912 Passos,Até que tentei relevar e viver o momento. Deixar de lado o sofrimento e tudo o que vejo. Não olh...,Dead Fish,129,0.7,Rock,Hardcore; Rock; Punk Rock; Regional; Rap; Ska; Pós-Punk,912 Passos Até que tentei relevar e viver o momento. Deixar de lado o sofrimento e tudo o que ve...,5
5071,Super-Heróis,Hoje é segunda-feira e decretamos feriado. Chamei Dom Paulo Coelho e saímos lado a lado. Lá na e...,Raul Seixas,215,14.0,Rock,Rock; Pop/Rock; MPB; Rock Alternativo; Clássico; Rockabilly; Psicodelia; Folk; Country; Blues; C...,Super-Heróis Hoje é segunda-feira e decretamos feriado. Chamei Dom Paulo Coelho e saímos lado a ...,5
6972,Surfando Karmas & Dna,"Quantas vezes eu estive. cara a cara com a pior metade?. A lembrança no espelho,. a esperança na...",Engenheiros do Hawaii,193,11.9,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Sertanejo; Progressivo; Rap; Country; Axé; Folk; For...,Surfando Karmas & Dna Quantas vezes eu estive. cara a cara com a pior metade?. A lembrança no es...,4
7142,Surfando Karmas & Dna,"Quantas vezes eu estive. cara a cara com a pior metade?. A lembrança no espelho,. a esperança na...",Engenheiros do Hawaii,193,11.9,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Sertanejo; Progressivo; Rap; Country; Axé; Folk; For...,Surfando Karmas & Dna Quantas vezes eu estive. cara a cara com a pior metade?. A lembrança no es...,4
3784,Lata Velha,"Te chamam de velharia,. Calhambeque, geringonça e carroça,. Mas sem você eu não vivo, Lata Velha...",Nx Zero,122,3.4,Rock,Rock,"Lata Velha Te chamam de velharia,. Calhambeque, geringonça e carroça,. Mas sem você eu não vivo,...",5
1238,Sofridos e Excluídos,Nosso sistema funciona de um jeito. Que os mais sofridos. São sempre excluídos. No poder. Abando...,CPM 22,114,3.2,Rock,Hardcore; Punk Rock; Rock; Pop/Rock; Hard Rock; Pop/Punk; Ska; Rock Alternativo; Axé; Heavy Meta...,Sofridos e Excluídos Nosso sistema funciona de um jeito. Que os mais sofridos. São sempre excluí...,5
4032,Muros,"Muros. Muros de pedra, muros de orgulho. Que nos separam, e nos envergonham. Muitos já caíram, o...",Oficina G3,134,2.3,Rock,Hard Rock; Pop/Rock; Rock; Gospel/Religioso; Heavy Metal; Hardcore; Progressivo; Piano Rock; Roc...,"Muros Muros. Muros de pedra, muros de orgulho. Que nos separam, e nos envergonham. Muitos já caí...",5
2647,Qualquer dia Desses,. Intr. G7. <<- 1a. parte 2 vezes. G. Qualquer dia desses. C. Eu vou tomar coragem. G7. Eu vou j...,Jota Quest,154,13.3,Rock,Pop/Rock; Pop; Rock; Romântico; Funk; Black Music; Trilha Sonora; Soul Music; Hip Hop; Rock Alte...,Qualquer dia Desses . Intr. G7. <<- 1a. parte 2 vezes. G. Qualquer dia desses. C. Eu vou tomar c...,8
6182,Anjo Exterminador,Hoje o ar vai nos faltar. Dois mil anos vão passar. Hoje a terra vai se abrir. Hoje o mar vai re...,Titãs,228,8.9,Rock,Pop/Rock; Rock; Hard Rock; MPB; Pop; World Music; Punk Rock; Heavy Metal; Pós-Punk; Rock Alterna...,Anjo Exterminador Hoje o ar vai nos faltar. Dois mil anos vão passar. Hoje a terra vai se abrir....,1
3068,Vida Bandida,"Chu..tou. A cara do cara caído, traiu. Traiu seu melhor, seu melhor amigo. Bateu, corrente, soco...",Lobão,112,1.3,Rock,Rock; MPB; Rock Alternativo; Punk Rock; Psicodelia; Progressivo; Pop/Rock; Axé; Romântico; Pós-P...,"Vida Bandida Chu..tou. A cara do cara caído, traiu. Traiu seu melhor, seu melhor amigo. Bateu, c...",6


In [35]:
letras_ag['grupo'].value_counts()

5    2353
4    1277
1    1113
3     862
2     729
8     600
6     266
7     260
0     220
Name: grupo, dtype: int64

In [36]:
# this loop transform the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-15:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : mundo, todo mundo, todo, baby, pra, dia, tudo, sol, tchu, ninguém, poeta, bem, noite, vida
1 : vai, gente, pra, tudo, vamos, dia, bem, faz, vida, ninguém, quer, sempre, tão, pode
2 : amor, pra, tudo, mim, coração, vida, quero, dor, vem, vou, sol, bem, viver, vai
3 : quero, sei, tanto, nada, aqui, quer, faz, pra, preciso, vou, cabeça, ficar, bom, vida
4 : bem, mim, sempre, vem, tudo, pra, quero, tão, sei, assim, vezes, nada, vejo, sinto
5 : pra, tempo, tudo, dia, nada, nunca, vida, ter, mim, agora, ver, viver, coração, mundo
6 : onde, vida, lugar, pra, tudo, vida vida, pra onde, sei, saber, leve, vai, dia, deus, aqui
7 : mãe, you, hey, the, and, faz, pra, homem, hey hey, your, chuva, deus, demais, that
8 : vou, pra, quero, deus, vai, mim, tudo, sei, onde, hoje, aqui, deixar, vou deixar, mundo


In [37]:
pd.options.display.max_colwidth = 100
letras_ag.query('grupo == 2')[["SName", "Lyric", "Artist"]].sample(10)

,SName,Lyric,Artist
3509,Lábios Divididos (Part. Thiaguinho),Meu amor!. Se sou refém do vai e vem dos seus lábios. Se estou perdido no calor dos seus braços....,Maná
2675,Um Dia Pra Não Se Esquecer (Sunrise) Ft. Projota,"Hey irmão, hoje era pra ser. O dia que ela voltaria de vez. Um dia pra não se esquecer. Mas você...",Jota Quest
2789,Love Song,Pois nasci nunca vi Amor. e ouço d' el sempre falar.. Pero sei que me quer matar. Mas rogarei a ...,Legião Urbana
5687,A Estratégia do Caos,A superfície aparente do olhar. Esconde um mar de lágrimas e estórias. De onde sereias parecem c...,RPM
5883,Balada do Amor Inabalável,"Eu levo essa canção de amor dançante pra você lembrar de mim,. Seu coração lembrar de mim. Na co...",Skank
5557,Angústia Suprema,Eu imagino o seu olhar. Eu o procuro no tempo. A última noite entre nós. Angústia ao extremo. Su...,Rosa de Saron
2566,A Lua,"A lua que faz brilhar o teu olhar, meu amor. Eu fico aqui tentando encontrar. A amplitude dos so...",Jota Quest
1912,Moço,"Moço , vê se larga o osso. Com a vida no bolso. Procura um amor , não tem. Moço , é madrugada. A...",Erasmo Carlos
2509,Minha Casa,"Minha casa, minha vida, minha luta, minha dor. Herança de uma fortuna construída com muito amor....",Isabella Taviani
1419,Olhos Certos,"Tento te encontrar. Tanto pra dizer. Meu amor, tudo bem. Sorte de nós dois. Quero te fazer feliz...",Detonautas


In [38]:
pd.options.display.max_colwidth = 100
letras_ag.query('grupo == 2')[["SName", "Lyric", "Artist"]].sample(10)

,SName,Lyric,Artist
2835,Soul Parsifal,Ninguém vai me dizer o que sentir. Meu coração está desperto. É sereno o nosso amor. E santo est...,Legião Urbana
7519,Tudo Com Você,Quero te conquistar. Um pouco mais e mais. A cada dia. Satisfazer tua vontade. Também me sacia. ...,Lulu Santos
5147,Outra Vez o Amor,A culpa é sua por não estarmos juntos. Você tem medo e não quer tocar no assunto. Ainda te afeto...,Reação Em Cadeia
4249,"Um Amor, Um Lugar",O meu amor é teu. O meu desejo é meu. O teu silêncio é um véu. O meu inferno é o céu. Pra quem n...,Os Paralamas do Sucesso
3132,Máscara Negra,"Máscara Negra. Los Hermanos. Composição: Zé Keti. Quanto riso, ah, quanta alegria. Mais de mil p...",Los Hermanos
5211,Caso Sério,Eu fico pensando em nós dois. Cada um na sua. Perdidos na cidade nua. Empapuçados de amor. Numa ...,Rita Lee
204,O Dia Em Que Você Me Salvou,O telefone tocou. Foi tudo muito rápido. Sua fala inesperada. Sou de um modo perfeito. Teria sid...,Barão Vermelho
1838,De Tanto Amor,"Ah ! Eu vim aqui amor só pra me despedir. E as últimas palavras desse nosso amor, você vai ter. ...",Erasmo Carlos
151,Codinome Beija-flor,Pra que mentir. Fingir que perdoou. Tentar ficar amigos sem rancor. A emoção acabou. Que coincid...,Barão Vermelho
5042,Planos de Papel,Deus eu passo os sete dias úteis. traçando nove dias fúteis. fazendo planos de papel. em quartos...,Raul Seixas


In [39]:
for g in range(0, 9):
    print("\n-----\nGrupo {}:".format(g))
    print(letras_ag.query('grupo == {}'.format(g))["Artist"].value_counts()[0:10])
    print("-----")


-----
Grupo 0:
Lulu Santos              28
Engenheiros do Hawaii    18
Barão Vermelho           15
Jota Quest               14
Blitz                    12
Lobão                    10
Cássia Eller             10
Velhas Virgens            9
Rita Lee                  8
Charlie Brown Jr          7
Name: Artist, dtype: int64
-----

-----
Grupo 1:
Lulu Santos              70
Engenheiros do Hawaii    64
Raimundos                49
Charlie Brown Jr         49
Jota Quest               46
Skank                    46
Velhas Virgens           39
Fresno                   32
Raul Seixas              30
Legião Urbana            29
Name: Artist, dtype: int64
-----

-----
Grupo 2:
Jota Quest                 60
Lulu Santos                56
Erasmo Carlos              39
Skank                      34
Rita Lee                   29
Cássia Eller               28
Cazuza                     27
Los Hermanos               26
Os Paralamas do Sucesso    25
Barão Vermelho             23
Name: Artist, dtype: int64

In [40]:
letras_ag.query('Artist == "Raça Negra"')[['grupo', 'SName']].groupby("grupo").count()

,SName
grupo,


In [41]:
letras_embedded = TSNE(n_components=2, verbose=1, perplexity=45).fit_transform(vec_text)

letras_ag = letras_ag.assign(tsnel = letras_embedded[:,0], tsne2 = letras_embedded[:,1])

alt.Chart(letras_ag.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = "tsnel",
    y = "tsne2",
    color = "grupo:N",
    tooltip = ["Artist", "SName"]
).interactive()

[t-SNE] Computing 136 nearest neighbors...
[t-SNE] Indexed 7680 samples in 0.003s...


c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computed neighbors for 7680 samples in 3.041s...
[t-SNE] Computed conditional probabilities for sample 1000 / 7680
[t-SNE] Computed conditional probabilities for sample 2000 / 7680
[t-SNE] Computed conditional probabilities for sample 3000 / 7680
[t-SNE] Computed conditional probabilities for sample 4000 / 7680
[t-SNE] Computed conditional probabilities for sample 5000 / 7680
[t-SNE] Computed conditional probabilities for sample 6000 / 7680
[t-SNE] Computed conditional probabilities for sample 7000 / 7680
[t-SNE] Computed conditional probabilities for sample 7680 / 7680
[t-SNE] Mean sigma: 0.546356
[t-SNE] KL divergence after 250 iterations with early exaggeration: 118.436821
[t-SNE] KL divergence after 1000 iterations: 3.745294


c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

# Questão 2 e 3
No lugar do PCA foi utilizado o TruncateSVD porque tem um problema "PCA does not support sparse input. See TruncatedSVD for a possible alternative"

In [50]:
# TF - IDF vectorizer
tfv = TfidfVectorizer(
    min_df = 5,
    max_df = 0.9,
    max_features = None,
    stop_words = stop_words,
    ngram_range = (1, 3)
)

# transform
vec_text = tfv.fit_transform(clean_lyrics)

words = tfv.get_feature_names()

len(words)

22581

k-Means sem o truncate

In [51]:
# setup kmeans clustering
kmeans = MiniBatchKMeans(n_clusters=9, init_size=1024, batch_size=2048, random_state=20)

# fit the data
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)

In [52]:
print("Coeficiente de Silhueta K-Means: %0.3f" % metrics.silhouette_score(vec_text, labels, metric='euclidean'))

Coeficiente de Silhueta K-Means: 0.002


In [58]:
pca = PCA(n_components=2)
PCA_X = pca.fit_transform(vec_text)

TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.

In [53]:
#pca = PCA(n_components=2)
#PCA_X = pca.fit_transform(vec_text)

svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
X_truncated  = svd.fit_transform(vec_text)

K-Means com o X Truncated

In [56]:
# setup kmeans clustering
kmeans = MiniBatchKMeans(n_clusters=9, init_size=1024, batch_size=2048, random_state=20)

# fit the data
kmeans.fit(X_truncated)
labels = kmeans.predict(X_truncated)



In [57]:
print("Coeficiente de Silhueta K-Means com o TruncateSVD: %0.3f" % metrics.silhouette_score(X_truncated, labels, metric='euclidean'))

Coeficiente de Silhueta K-Means com o TruncateSVD: 0.189


Questão 3: Compare a qualidade dos grupos com e sem PCA usando o coeficiente de silhueta.

Comparando a qualidade dos grupos percebe-se que o coeficiente de silhueta é melhor (0.0012 contra 0.189) quando aplicado a redução de dimensionalidade